# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [ ]:
#Command used to import the json file to our database. Note: The command was 
# run inside the folder where the json file is located

#mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

In [28]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'local', 'uk_food']


In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [11]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [12]:
# review a document in the establishments collection
pprint(db['establishments'].find_one())

{'AddressLine1': '3 The Droveway',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Boodles',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.638929767669,
 'FHRSID': 1349980,
 'LocalAuthorityBusinessID': 'PI/000084064',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DH',
 'RatingDate': '2021-06-14T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('669ad4dbd8e606a7bdf4975f'),
 'geocode': {'latitude': '51.150879', 'longitude': '1.378536'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1349980',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T00

In [13]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [23]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}


In [24]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

InsertOneResult(ObjectId('669aebcdfd855706176827e4'), acknowledged=True)

In [25]:
# Check that the new restaurant was inserted
query = {'BusinessName': 'Penang Flavours'}
results = establishments.find(query)
for result in results:
    pprint(result)

# Delete restaurant from collection
#establishments.delete_one(query)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('669aebcdfd855706176827e4'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [31]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType':"Restaurant/Cafe/Canteen"}
fields = {'BusinessType':1, 'BusinessTypeID':1}
limit= 1
results = list(establishments.find(query, fields).limit(limit))
results_df = pd.DataFrame(results)
results_df =  results_df[results_df.columns[1:]]
results_df

,BusinessType,BusinessTypeID
0,Restaurant/Cafe/Canteen,1


3. Update the new restaurant with the `BusinessTypeID` you found.

In [34]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one(
    {'BusinessName':'Penang Flavours'},
    {'$set':
        {'BusinessTypeID':1}
    }
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [35]:
# Confirm that the new restaurant was updated
query = {'BusinessName': 'Penang Flavours'}
fields = {'BusinessName':1, 'BusinessType':1, 'BusinessTypeID':1}
results = establishments.find(query, fields)
for result in results:
    pprint(result)

{'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('669aebcdfd855706176827e4')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [50]:
# Find how many documents have LocalAuthorityName as "Dover"
count = 0
query = {'LocalAuthorityName':'Dover'}
results = establishments.find(query)
for result in results:
    count = count + 1
print(f'Number of documents with LocalAuthorityName as Dover is: {count}')

Number of documents with LocalAuthorityName as Dover is: 994


In [53]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many(query)

DeleteResult({'n': 994, 'ok': 1.0}, acknowledged=True)

In [57]:
# Check if any remaining documents include Dover
dover_count = establishments.count_documents({'LocalAuthorityName':'Dover'})
print(f'Number of documents with LocalAuthorityName as Dover is: {dover_count}')

Number of documents with LocalAuthorityName as Dover is: 0


In [55]:
# Check that other documents remain with 'find_one'
pprint(db['establishments'].find_one())

{'AddressLine1': 'The Ship',
 'AddressLine2': '23 The Stade',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Ships Galley',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.912144538092,
 'FHRSID': 987206,
 'LocalAuthorityBusinessID': 'PI/000075655',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6AB',
 'RatingDate': '2019-07-03T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('669ad4dbd8e606a7bdf49a47'),
 'geocode': {'latitude': '51.08084', 'longitude': '1.188537'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/987206',
            'rel': 'self'}],
 'meta': {'dataSource': None,
        

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [151]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [{'$set':
                                    {'geocode.longitude':{'$toDecimal':'$geocode.longitude'}}},
                                {'$set':
                                    {'geocode.latitude':{'$toDecimal':'$geocode.latitude'}}}
                               ]
                          )

UpdateResult({'n': 38786, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [141]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [146]:
# Change the data type from String to Integer for RatingValue
#Count establishments with latitude and longitude set to decimal before changing data types
count_rating = 0
query = {'RatingValue':{'$type':'int'}}
results = establishments.find(query,fields)
for result in results:
    count_rating = count_rating +1
print(f'There are {count_rating} establishments with rating value set as integer before changing the data type /n')

# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [{'$set':
                                    {'RatingValue':{'$toInt':'$RatingValue'}}}
                               ]
                          )

There are 0 establishments with rating value set as integer before changing the data type /n


UpdateResult({'n': 38786, 'nModified': 34695, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [147]:
#Count establishments with latitude and longitude set to decimal after changing data types
count_decimal = 0
query = {'geocode.latitude':{'$type':'decimal'}, 'geocode.longitude':{'$type':'decimal'}}
results = establishments.find(query_latlon,fields)
for result in results:
    count_decimal = count_decimal +1
print(f'There are {count_decimal} establishments with latitude and longitude set as decimal after changing the data type')

# Check that the coordinates and rating value are now numbers
count_rating = 0
query = {'RatingValue':{'$type':'int'}}
results = establishments.find(query,fields)
for result in results:
    count_rating = count_rating +1
print(f'There are {count_rating} establishments with rating value set as integer before changing the data type')


There are 38786 establishments with latitude and longitude set as decimal after changing the data type
There are 34694 establishments with rating value set as integer before changing the data type
